In [36]:
from connections.client import BinanceClient
from config.settings import BINANCE_API_KEY, BINANCE_SECRET_KEY, USE_TESTNET
from data.fetch import BinanceDataFetcher,BinanceOptionsFetcher

# import importlib
# import data.fetch
# importlib.reload(data.fetch)

In [37]:
# Test settings
print("API Key:", BINANCE_API_KEY)
print("Testnet Mode:", USE_TESTNET)

# Test the BinanceClient
client = BinanceClient()
print(client.test_connection())


API Key: FdEFwUP7BbC4y4GshdHi3SuGaWUTmqABFUSD1J3tySAGxsxGAumA9zaeR5wYI9RVFdEFwUP7BbC4y4GshdHi3SuGaWUTmqABFUSD1J3tySAGxsxGAumA9zaeR5wYI9RV
Testnet Mode: True
Initialized Binance Client (Testnet: True)
Connection successful: {}
{}


In [38]:
import os
print(os.getcwd())

/home/ec2-user/BinanceEDA


In [39]:
import sys
print(sys.path)


['/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/home/ec2-user/.local/lib/python3.9/site-packages', '/home/ec2-user/BinanceEDA', '/usr/lib64/python3.9/site-packages', '/usr/lib/python3.9/site-packages']


In [40]:


# Initialize the data fetcher
data_fetcher = BinanceDataFetcher()

# Fetch historical candlestick data
historical_data = data_fetcher.get_historical_klines(limit=5)
print("Historical Data:", historical_data)

# Fetch current market price
current_price = data_fetcher.get_current_price()
print("Current Price:", current_price)

# Fetch recent trades
recent_trades = data_fetcher.get_recent_trades(limit=5)
print("Recent Trades:", recent_trades)


INFO:data.fetch:Fetching historical klines for BTCUSDT with interval 1m and limit 5


Initialized Binance Client (Testnet: True)


INFO:data.fetch:Fetching current price for BTCUSDT


Historical Data: [[1735398720000, '94540.39000000', '94619.63000000', '94520.99000000', '94583.10000000', '0.08868000', 1735398779999, '8386.20336670', 39, '0.07684000', '7267.04493790', '0'], [1735398780000, '94521.02000000', '94602.33000000', '94521.02000000', '94602.33000000', '0.05572000', 1735398839999, '5270.40034880', 23, '0.04942000', '4674.91109780', '0'], [1735398840000, '94602.33000000', '94605.44000000', '94592.51000000', '94605.44000000', '0.06416000', 1735398899999, '6069.47413780', 34, '0.06091000', '5762.04848030', '0'], [1735398900000, '94605.44000000', '94624.90000000', '94592.27000000', '94624.90000000', '0.07306000', 1735398959999, '6912.05136620', 31, '0.04541000', '4296.56896970', '0'], [1735398960000, '94592.27000000', '94624.90000000', '94592.27000000', '94624.90000000', '0.00146000', 1735399019999, '138.11776620', 2, '0.00040000', '37.84996000', '0']]


INFO:data.fetch:Fetching recent trades for BTCUSDT with limit 5


Current Price: 94608.63000000
Recent Trades: [{'id': 5844910, 'price': '94624.89000000', 'qty': '0.00158000', 'quoteQty': '149.50732620', 'time': 1735398959509, 'isBuyerMaker': False, 'isBestMatch': True}, {'id': 5844911, 'price': '94624.90000000', 'qty': '0.00186000', 'quoteQty': '176.00231400', 'time': 1735398959509, 'isBuyerMaker': False, 'isBestMatch': True}, {'id': 5844912, 'price': '94592.27000000', 'qty': '0.00106000', 'quoteQty': '100.26780620', 'time': 1735398963153, 'isBuyerMaker': True, 'isBestMatch': True}, {'id': 5844913, 'price': '94624.90000000', 'qty': '0.00040000', 'quoteQty': '37.84996000', 'time': 1735398963415, 'isBuyerMaker': False, 'isBestMatch': True}, {'id': 5844914, 'price': '94608.63000000', 'qty': '0.00423000', 'quoteQty': '400.19450490', 'time': 1735398974412, 'isBuyerMaker': False, 'isBestMatch': True}]


In [42]:
# Initialize the options fetcher
options_fetcher = BinanceOptionsFetcher(api_key=BINANCE_API_KEY, api_secret=BINANCE_SECRET_KEY)

# Define the option symbol
option_symbol = "BTC-250328-90000-P"  # Replace with a valid symbol if necessary



In [45]:
# Fetch and print option info
print(f"Fetching option info for {option_symbol}...")

try:
    option_info = options_fetcher.get_option_info(option_symbol)
    print("Option Info Retrieved Successfully:")
    print(option_info)
except Exception as e:
    print(f"Error fetching option info: {e}")


INFO:data.fetch:Making GET request to https://vapi.binance.com/vapi/v1/optionInfo with params: {'symbol': 'BTC-250328-90000-P'}


Fetching option info for BTC-250328-90000-P...


ERROR:data.fetch:Request failed: Expecting value: line 1 column 1 (char 0)
ERROR:data.fetch:Error fetching option info for BTC-250328-90000-P: Request failed: Expecting value: line 1 column 1 (char 0)


Error fetching option info: Request failed: Expecting value: line 1 column 1 (char 0)
